In [ ]:
%pip install numpy pandas seaborn matplotlib plotly scikit-learn nltk wordcloud pyspellchecker beautifulsoup4 prettytable

### Loading of Data

In [2]:
import pandas as pd
df = pd.read_csv(r'C:\Users\tiled\OneDrive\Desktop\DataSet\IMDB-Dataset.csv',encoding='latin-1')

In [3]:
df.head(10)["Reviews"]

0    *Disclaimer: I only watched this movie as a co...
1    I am writing this in hopes that this gets put ...
2    Really, I could write a scathing review of thi...
3    If you saw the other previous spoof movies by ...
4    This movie I saw a day early for free and I st...
5    Honestly, what is wrong with you, Hollywood? N...
6    I was given a free ticket to this film; so I c...
7    OK, so "Disastrous" isn't an imaginative barb ...
8    Jason Friedberg and Aaron Seltzer, the way eve...
9    Honestly the worst movie ever made. Theatre fu...
Name: Reviews, dtype: object

### Customizing Stopwords for Enhanced Text Preprocessing

*Enhancing the default stopword list by adding domain-specific words and removing negations to improve text analysis accuracy.*

In [4]:
from nltk.corpus import stopwords
import nltk

stop_words = stopwords.words('english')
nltk.download('averaged_perceptron_tagger')

new_stopwords = [
"would", "shall", "could", "might", "movie", "movies", "film", "films",
"cinema", "director", "directors", "actor", "actors", "actress", "actresses",
"cast", "screenplay", "plot", "story", "character", "characters", "scene",
"scenes", "soundtrack", "soundtracks","sequel", "prequel", "adaptation", "trailer", "genre", "genres", "release",
"theaters", "theatre", "performance", "performances", "acting", "blockbuster",
"indie", "oscars", "oscar", "award-winning", "box", "office", "premiere",
"rating", "ratings", "reviews", "animation", "cinematography", "editing", "script", "narrative",
"direction", "score", "soundtrack"
]

stop_words.extend(new_stopwords)
stop_words.remove("not")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\tiled\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
stop_words=set(stop_words)
print(len(stop_words))

233


### Data Cleaning and Text Preprocessing Pipeline

*Comprehensive functions for cleaning and preparing text data, including removing special characters, URLs, stopwords, and expanding contractions to enhance the quality of the dataset.*

### Correction of Typos

*Written text often contains errors, such as “Fen” instead of “Fan.” To rectify these errors, a dictionary is employed to map words to their correct forms based on similarity.*

In [6]:
from spellchecker import SpellChecker

spell = SpellChecker()

def correct_typos(content):
    """
    Parameters:
    - content (str): The input text to be corrected.

    Returns:
    - str: The text with corrected typos.
    """
    try:
        words = content.split()
        corrected_words = []
        misspelled = spell.unknown(words)

        for word in words:
            if word in misspelled:
                corrected_word = spell.correction(word)
                corrected_words.append(corrected_word)
            else:
                corrected_words.append(word)
        
        corrected_content = ' '.join(corrected_words)
        return corrected_content
    except Exception as e:
        print(f"Error in correcting typos: {e}")
        return content

### Mapping and Replacement

*This involves mapping words to standardized language equivalents. For instance, words like “b4” and “ttyl,” commonly understood by humans as “before” and “talk to you later,” pose challenges for machines. Normalization entails mapping such words to their standardized counterparts.*

In [7]:
import re

def load_mapping_dictionary():
    """
    Returns:
    - dict: A dictionary mapping non-standard words to standardized words.
    """
    mapping_dict = {
        "b4": "before",
        "ttyl": "talk to you later",
        "u": "you",
        "r": "are",
        "lol": "laughing out loud",
        "idk": "i do not know",
        "btw": "by the way",
        "omg": "oh my god",
        "imo": "in my opinion",
        "np": "no problem",
        "gr8": "great",
        "l8r": "later",
        "gtg": "got to go",
        "thx": "thanks",
        "pls": "please",
        "plz": "please",
        "bc": "because",
        "cuz": "because",
        "y'all": "you all",
        "luv": "love",
        "wanna": "want to",
        "gonna": "going to",
        "hafta": "have to",
        "kinda": "kind of",
        "sorta": "sort of",
        "gimme": "give me",
        "lemme": "let me",
        "whatcha": "what are you",
        "whaddaya": "what do you",
    }
    return mapping_dict

def normalize_text(content, mapping_dict):
    """
    Parameters:
    - content (str): The input text to be normalized.
    - mapping_dict (dict): A dictionary mapping non-standard words to standardized words.
    
    Returns:
    - str: The normalized text.
    """
    try:
        words = content.split()
        normalized_words = []
        
        for word in words:
            clean_word = re.sub(r'[^\w\s]', '', word.lower())
            if clean_word in mapping_dict:
                replacement = mapping_dict[clean_word]
                normalized_words.append(replacement)
            else:
                normalized_words.append(word)
        
        normalized_content = ' '.join(normalized_words)
        return normalized_content
    except Exception as e:
        print(f"Error in normalizing text: {e}")
        return content

### Expanding Contractions

*Convert contractions like “don’t” to “do not” or “I’ll” to “I will.”*

In [8]:
contractions = {
        r"won't": "will not",
        r"can't": "cannot",
        r"i'm": "i am",
        r"ain't": "is not",
        r"let's": "let us",
        r"ma'am": "madam",
        r"shan't": "shall not",
        r"n't": " not",
        r"'re": " are",
        r"'s": " is",
        r"'d": " would",
        r"'ll": " will",
        r"'ve": " have",
        r"'m": " am",
        r"he's": "he is",
        r"she's": "she is",
        r"it's": "it is",
        r"that's": "that is",
        r"there's": "there is",
        r"who's": "who is",
        r"what's": "what is",
        r"where's": "where is",
        r"when's": "when is",
        r"why's": "why is",
        r"how's": "how is",
        r"would've": "would have",
        r"could've": "could have",
        r"should've": "should have",
        r"might've": "might have",
        r"must've": "must have",
        r"wouldn't": "would not",
        r"couldn't": "could not",
        r"shouldn't": "should not",
        r"mightn't": "might not",
        r"mustn't": "must not",
        r"don't": "do not",
        r"doesn't": "does not",
        r"didn't": "did not",
        r"hasn't": "has not",
        r"haven't": "have not",
        r"hadn't": "had not",
        r"can't've": "cannot have",
        r"shan't've": "shall not have",
        r"wouldn't've": "would not have",
        r"couldn't've": "could not have",
        r"shouldn't've": "should not have",
        r"mightn't've": "might not have",
        r"mustn't've": "must not have",
        r"i'd": "i would",
        r"i'll": "i will",
        r"i've": "i have",
        r"i'm": "i am",
        r"you'd": "you would",
        r"you'll": "you will",
        r"you've": "you have",
        r"you're": "you are",
        r"he'd": "he would",
        r"he'll": "he will",
        r"he's": "he is",
        r"she'd": "she would",
        r"she'll": "she will",
        r"she's": "she is",
        r"it'd": "it would",
        r"it'll": "it will",
        r"it's": "it is",
        r"they'd": "they would",
        r"they'll": "they will",
        r"they're": "they are",
        r"we'd": "we would",
        r"we'll": "we will",
        r"we're": "we are",
        r"there'd": "there would",
        r"that'd": "that would",
        r"who'd": "who would",
        r"who'll": "who will",
        r"who're": "who are",
        r"what've": "what have",
        r"where've": "where have",
        r"when've": "when have",
        r"why've": "why have",
        r"how've": "how have",
        r"lets": "let us",
        r"lets'": "let us",
        r"gonna": "going to",
        r"wanna": "want to",
        r"gotta": "got to",
        r"oughtn't": "ought not",
        r"needn't": "need not",
        r"daren't": "dare not",
        r"maam": "madam",
        r"gimme": "give me",
        r"lemme": "let me",
        r"whatcha": "what are you",
        r"whaddaya": "what do you",
        r"gimme": "give me",
        r"gonna": "going to",
        r"gotta": "got to",
        r"hafta": "have to",
        r"wanna": "want to",
        r"ain't": "am not",
        r"y'all": "you all",
        r"coulda": "could have",
        r"woulda": "would have",
        r"shoulda": "should have",
        r"'bout": "about",
        r"'til": "until",
        r"can't've": "cannot have",
        r"could've": "could have",
        r"might've": "might have",
        r"must've": "must have",
        r"should've": "should have",
        r"would've": "would have",
        r"you'd've": "you would have",
    }

def contraction_expansion(content):
    """
    Parameters:
    - content (str): The input text.

    Returns:
    - str: Text with expanded contractions.
    """
    
    for contraction, expanded in contractions.items():
        pattern = re.compile(contraction, flags=re.IGNORECASE)
        content = pattern.sub(expanded, content)
    return content

### Removing Accents and Diacritics
*Sometimes, people use accented characters like é, ö, etc. to signify emphasis on a particular letter during pronunciation. Normalize text by removing accents and diacritical marks from characters.*

In [9]:
import unicodedata

def remove_accents(content):
    """
    Parameters:
    - content (str): The input text to be normalized.
    
    Returns:
    - str: The text with accents and diacritics removed.
    """
    try:
        normalized = unicodedata.normalize('NFKD', content)
        without_accents = ''.join([c for c in normalized if not unicodedata.combining(c)])
        return without_accents
    except Exception as e:
        print(f"Error in removing accents: {e}")
        return content

### Removing Extra Whitespace

*Normalize text by removing extra spaces and leading/trailing spaces*

In [10]:
def remove_extra_whitespace(text):
    """
    Parameters:
    - text (str): The input text to be normalized.
    
    Returns:
    - str: The text with extra spaces removed.
    """
    return ' '.join(text.split())

### Eliminating HTML Tags

*In cases where raw text originates from sources such as web scraping or screen capture, it often carries along HTML tags. These tags introduce unwanted noise and contribute little to the comprehension and analysis of the text. Therefore, it becomes necessary to strip them.*

In [11]:
from bs4 import BeautifulSoup

def remove_html_tags(text):
    """
    Parameters:
    - text (str): The input text containing HTML tags.
    
    Returns:
    - str: The text with HTML tags removed.
    """
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

### Handling URLs

*Frequently, individuals include URLs, particularly in social media content, to supplement context with additional information. However, URLs tend to vary across samples and can be considered noise.*

In [12]:
import re 

def remove_url(text):
    return re.sub(r"(https|http)?:\S*", "", text)

### Case Standardising and Removing Special Characters

*Special characters are non-alphanumeric characters. The characters like %,$,&, etc are special. In most NLP tasks, these characters add no value to text understanding and induce noise into algorithms. We can use regular expressions to remove special characters.*

In [13]:
def remove_special_character(content):
    content = content.lower()
    return re.sub(r'\W+',' ', content)

### Removing Stopwords

*In most cases, stopwords like I, am, me, etc. don’t add any information that can help in modeling. Keeping them in the text introduces unnecessary noise and can significantly increase the dimensionality of feature vectors, which can negatively impact both computation cost and model accuracy.*

In [14]:
def remove_stopwords(content):
    clean_data = []
    for i in content.split():
        if i.strip().lower() not in stop_words and i.strip().lower().isalpha():
            clean_data.append(i.strip().lower())
    return " ".join(clean_data)

In [15]:
def data_cleaning(content):
    # content = correct_typos(content)
    mapping_dict = load_mapping_dictionary()
    content = remove_extra_whitespace(content)
    content = normalize_text(content, mapping_dict)
    content = contraction_expansion(content)
    content = remove_url(content)
    content = remove_accents(content)
    # content = remove_html_tags(content)
    content = remove_special_character(content)
    content = remove_stopwords(content)
    return content

### Applying Data Cleaning to Reviews and Displaying Results

In [16]:
%%time
pd.options.display.max_colwidth = 1000
df['Reviews_clean']=df['Reviews'].apply(data_cleaning)
df.head(5)

CPU times: total: 1min 21s
Wall time: 4min 7s


,Ratings,Reviews,Movies,Resenhas,Reviews_clean
0,1.0,"*Disclaimer: I only watched this movie as a conditional agreement. And I see films for free. I wouldn't be caught dead giving my hard earned money to these idiots.Well, to explain the depth of this 'film', I could write my shortest review, ever. Don't see this movie. It is by far the stupidest, lamest, most lazy, and unbelievably UNFUNNY movie I have ever seen. It is a total disaster. But since my hatred for this movie, and the others like it, extends far beyond one viewing, I think I'll go on for a bit.I don't know any of the people in the movie besides Carmen Electra, Vanessa Minnillo, and Kim Kardashian, but it doesn't matter. They're all horrible, though I think that was the point. The editing is flat out horrible, and possibly blatant continuity errors make this crapfast even crappier than I thought it would be. Now I know that these films are not supposed to be serious at all, but come on, it's film-making 101 that if someone gets a minor facial cut, it should be there in the...",Disaster Movie,"* IsenÃ§Ã£o de responsabilidade: eu sÃ³ assisti esse filme como um acordo condicional. E eu vejo filmes de graÃ§a. Eu nÃ£o seria pego morto dando meu dinheiro suado a esses idiotas. Bem, para explicar a profundidade desse 'filme', eu poderia escrever minha crÃ­tica mais curta de todos os tempos. NÃ£o vÃª este filme. Ã de longe o filme mais estÃºpido, lamenta, preguiÃ§oso e inacreditavelmente UNFUNNY que eu jÃ¡ vi. Ã um desastre total. Mas como o meu Ã³dio por este filme e por outros, se estende muito alÃ©m de uma exibiÃ§Ã£o, acho que vou continuar um pouco. NÃ£o conheÃ§o nenhuma das pessoas do filme alÃ©m de Carmen Electra, Vanessa Minnillo, e Kim Kardashian, mas isso nÃ£o importa. Eles sÃ£o todos horrÃ­veis, embora eu ache que esse seja o ponto. A ediÃ§Ã£o Ã© horrÃ­vel e, possivelmente, erros de continuidade flagrantes tornam essa porcaria ainda mais horrÃ­vel do que eu pensava. Agora eu sei que esses filmes nÃ£o devem ser sÃ©rios, mas vamos lÃ¡, Ã© o cinema 101 que se alguÃ©m f...",disclaimer watched conditional agreement see free not caught dead giving hard earned money idiots well explain depth write shortest review ever not see far stupidest lamest lazy unbelievably unfunny ever seen total disaster since hatred others like extends far beyond one viewing think go bit not know people besides carmen electra vanessa minnillo kim kardashian not matter horrible though think point flat horrible possibly blatant continuity errors make crapfast even crappier thought know not supposed serious come making someone gets minor facial cut next shot someone gets cut sword blood least cut though since narnia get away give disaster pass jokes thoughtless mindless physical gags obviously take popular last year late well including best picture nominees know saddest thing stupid not care much money make many cameos sorry ass excuses taking away jobs writers truly deserve attention lionsgate thought better taste ashamed making kind crap jason friedberg aaron seltzer burn hell g...
1,1.0,"I am writing this in hopes that this gets put over the previous review of this ""film"". How anyone can find this slop entertaining is completely beyond me. First of all a spoof film entitled ""Disaster Movie"", should indeed be a spoof on disaster films. Now I have seen 1 (yes count them, 1) disaster film being spoofed, that being ""Twister"". How does Juno, Iron Man, Batman, The Hulk, Alvin and the Chipmunks, Amy Winehouse, or Hancock register as Disaster films? Selzterwater and Failburg once again have shown that they lack any sort of writing skill and humor. Having unfortunately been tortured with Date Movie and Epic Movie I know exactly what to expect from these two...no plot, no jokes just bad references and cheaply remade scenes from other films. Someone should have informed them that satire is more than just copy and paste from one film to another, though I shouldn't say that because some of these ac

## Feature Engineering

In [17]:
df['Label'] = df['Ratings'].apply(lambda x: '1' if x >= 7 else ('0' if x<=4 else '2'))

df=df[df.Label<'2']
data=df[['Reviews_clean','Label']]

data['Label'].value_counts()

Label
0    60000
1    60000
Name: count, dtype: int64

### Lemmatization

In [18]:
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize  

nltk.download('wordnet')
nltk.download('punkt')

class LemmaTokenizer(object):
    def __init__(self):
        self.wordnetlemma = WordNetLemmatizer()
    
    def __call__(self, reviews):
        """
        Lemmatizes a list of tokens to their base form.

        Parameters:
        - tokens (list): A list of words to be lemmatized.

        Returns:
        - list: A list of lemmatized words.
        """
        return [self.wordnetlemma.lemmatize(word) for word in word_tokenize(reviews)]

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tiled\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tiled\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Text Vectorization with CountVectorizer and TF-IDF

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from prettytable import PrettyTable

In [20]:
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42, shuffle=True)

In [21]:
y_train = train_data['Label']
y_test = test_data['Label']

In [22]:
import warnings
warnings.filterwarnings('ignore')

In [23]:
def vectorize_text(train_data, test_data, tokenizer, vectorizer_type='count', ngram_range=(1, 1), min_df=10, max_features=500):
    """
    Vectorizes text data using specified vectorizer type.

    Parameters:
    - train_data (pd.Series): The training text data to be vectorized.
    - test_data (pd.Series): The test text data to be vectorized.
    - tokenizer (callable): A tokenizer function or class instance.
    - vectorizer_type (str): Type of vectorizer to use ('count' or 'tfidf').
    - ngram_range (tuple): The range of n-grams to consider.
    - min_df (int): Minimum document frequency for terms.
    - max_features (int): Maximum number of features to consider.

    Returns:
    - np.ndarray: The vectorized training text data.
    - np.ndarray: The vectorized test text data.
    - Vectorizer: The fitted vectorizer instance (CountVectorizer or TfidfVectorizer).
    """
    if vectorizer_type == 'count':
        vectorizer = CountVectorizer(analyzer="word", 
                                     tokenizer=tokenizer, 
                                     ngram_range=ngram_range, 
                                     min_df=min_df,
                                     max_features=max_features)
    elif vectorizer_type == 'tfidf':
        vectorizer = TfidfVectorizer(analyzer="word", 
                                     tokenizer=tokenizer, 
                                     ngram_range=ngram_range, 
                                     min_df=min_df,
                                     max_features=max_features)
    else:
        raise ValueError("vectorizer_type must be either 'count' or 'tfidf'")

    x_train_vectorized = vectorizer.fit_transform(train_data).toarray()
    x_test_vectorized = vectorizer.transform(test_data).toarray()

    return x_train_vectorized, x_test_vectorized, vectorizer

In [24]:
def train_logistic_regression(x_train, y_train, x_test, y_test, vectorizer, top_n_features=50):
    """
    Trains a Logistic Regression model and extracts the top N important features.

    Parameters:
    - x_train (np.ndarray): The vectorized training data.
    - y_train (pd.Series): The training labels.
    - x_test (np.ndarray): The vectorized test data.
    - y_test (pd.Series): The test labels.
    - vectorizer: The fitted vectorizer instance (CountVectorizer or TfidfVectorizer).
    - top_n_features (int): The number of top features to display.

    Returns:
    - float: The accuracy score of the model on the test data.
    - PrettyTable: A table of the top N important features and their coefficients.
    """
    logistic_regression_model = LogisticRegression()
    logistic_regression_model.fit(x_train, y_train)
    accuracy = logistic_regression_model.score(x_test, y_test)

    coefficients = logistic_regression_model.coef_[0]
    important_features_table = PrettyTable(["Feature", "Coefficient"])

    sorted_features = sorted(zip(vectorizer.get_feature_names_out(), coefficients), 
                             key=lambda x: abs(x[1]), reverse=True)

    for feature, coefficient in sorted_features[:top_n_features]:
        important_features_table.add_row([feature, coefficient])

    return accuracy, important_features_table

### Testing Logistic Regression with Count and TF-IDF Vectorizers: Unigrams

**Unigrams with CountVectorizer**

In [25]:
x_train_unigrams_count_vectorized, x_test_unigrams_count_vectorized, unigram_count_vectorizer = vectorize_text(
    train_data['Reviews_clean'], 
    test_data['Reviews_clean'], 
    tokenizer=LemmaTokenizer(), 
    vectorizer_type='count', 
    ngram_range=(1, 1), 
    min_df=10, 
    max_features=500
)

**Logistic Regression with Unigrams and CountVectorizer**

In [26]:
accuracy_unigrams_count, important_features_table_unigrams_count = train_logistic_regression(
    x_train_unigrams_count_vectorized, 
    y_train, 
    x_test_unigrams_count_vectorized, 
    y_test, 
    unigram_count_vectorizer, 
    top_n_features=50
)
important_features_table_unigrams_count

Feature,Coefficient
waste,-1.5262028005177193
worst,-1.2355923498425816
awful,-1.2085420339457122
unfortunately,-1.1153074102194014
boring,-1.0910975318382654
excellent,0.9719532415402448
perfect,0.9353773135045919
terrible,-0.9330287932480498
enjoyed,0.9255730908322781
poor,-0.9182509919769848


**Unigrams with TfidfVectorizer**

In [27]:
x_train_unigrams_tfidf_vectorized, x_test_unigrams_tfidf_vectorized, unigram_tfidf_vectorizer = vectorize_text(
    train_data['Reviews_clean'], 
    test_data['Reviews_clean'], 
    tokenizer=LemmaTokenizer(), 
    vectorizer_type='tfidf', 
    ngram_range=(1, 1), 
    min_df=10, 
    max_features=500
)

**Logistic Regression with Unigrams and TfidfVectorizer**

In [28]:
accuracy_unigrams_tfidf, important_features_table_unigrams_tfidf = train_logistic_regression(
    x_train_unigrams_tfidf_vectorized, 
    y_train, 
    x_test_unigrams_tfidf_vectorized, 
    y_test, 
    unigram_tfidf_vectorizer, 
    top_n_features=50
)
important_features_table_unigrams_tfidf

Feature,Coefficient
worst,-7.9723383455910835
waste,-7.624880469958248
awful,-6.810376409312876
boring,-6.3459830645281565
unfortunately,-6.087616349730546
perfect,5.934083405347333
terrible,-5.671844927093191
enjoyed,5.641929677443677
great,5.613390243009951
excellent,5.6010320775678455


### Testing Logistic Regression with Count and TF-IDF Vectorizers: Bigrams

**Bigrams with CountVectorizer**

In [29]:
x_train_bigrams_count_vectorized, x_test_bigrams_count_vectorized, bigram_count_vectorizer = vectorize_text(
    train_data['Reviews_clean'], 
    test_data['Reviews_clean'], 
    tokenizer=LemmaTokenizer(), 
    vectorizer_type='count', 
    ngram_range=(2, 2), 
    min_df=10, 
    max_features=500
)

**Logistic Regression with Bigrams and CountVectorizer**

In [30]:
accuracy_bigrams_count, important_features_table_bigrams_count = train_logistic_regression(
    x_train_bigrams_count_vectorized, 
    y_train, 
    x_test_bigrams_count_vectorized, 
    y_test, 
    bigram_count_vectorizer, 
    top_n_features=50
)
important_features_table_bigrams_count

Feature,Coefficient
definitely worth,2.536222023771126
highly recommended,2.4940551799892905
worst seen,-2.4642270440362974
not perfect,2.4231397806628947
pleasantly surprised,2.3793781442546273
well worth,2.149524640607187
highly recommend,2.096602004327463
waste money,-2.0514546697445493
redeeming quality,-2.0041719199106147
not worth,-1.9371119806855495


**Bigrams with TfidfVectorizer**

In [31]:
x_train_bigrams_tfidf_vectorized, x_test_bigrams_tfidf_vectorized, bigram_tfidf_vectorizer = vectorize_text(
    train_data['Reviews_clean'], 
    test_data['Reviews_clean'], 
    tokenizer=LemmaTokenizer(), 
    vectorizer_type='tfidf', 
    ngram_range=(2, 2), 
    min_df=10, 
    max_features=500
)

**Logistic Regression with Bigrams and TfidfVectorizer**

In [32]:
accuracy_bigrams_tfidf, important_features_table_bigrams_tfidf = train_logistic_regression(
    x_train_bigrams_tfidf_vectorized, 
    y_train, 
    x_test_bigrams_tfidf_vectorized, 
    y_test, 
    bigram_tfidf_vectorizer, 
    top_n_features=50
)
important_features_table_bigrams_tfidf

Feature,Coefficient
not perfect,5.057100988600406
pleasantly surprised,4.972249753417221
definitely worth,4.736471013767229
waste time,-4.543797151030226
not worth,-4.495432856928245
worst seen,-4.309412634808848
highly recommended,4.2589568748665885
well worth,4.136814921831428
one best,4.077068875457178
one worst,-4.04405182082838


### Testing Logistic Regression with Count and TF-IDF Vectorizers: Trigrams

**Trigrams with CountVectorizer**

In [33]:
x_train_trigrams_count_vectorized, x_test_trigrams_count_vectorized, trigram_count_vectorizer = vectorize_text(
    train_data['Reviews_clean'], 
    test_data['Reviews_clean'], 
    tokenizer=LemmaTokenizer(), 
    vectorizer_type='count', 
    ngram_range=(3, 3), 
    min_df=10, 
    max_features=500
)

**Logistic Regression with Trigrams and CountVectorizer**

In [34]:
accuracy_trigrams_count, important_features_table_trigrams_count = train_logistic_regression(
    x_train_trigrams_count_vectorized, 
    y_train, 
    x_test_trigrams_count_vectorized, 
    y_test, 
    trigram_count_vectorizer, 
    top_n_features=50
)
important_features_table_trigrams_count

Feature,Coefficient
can not save,-3.4392363820095335
highly recommend anyone,3.0423203248029265
definitely worth watch,2.9844651897982635
one worst seen,-2.8832693281093755
really wanted like,-2.8191540193844147
not worth time,-2.786776947199439
not enough save,-2.786754877795848
not really worth,-2.739557283334873
not good either,-2.5931068128524988
definitely worth watching,2.590533752210994


**Trigrams with TfidfVectorizer**

In [35]:
x_train_trigrams_tfidf_vectorized, x_test_trigrams_tfidf_vectorized, trigram_tfidf_vectorizer = vectorize_text(
    train_data['Reviews_clean'], 
    test_data['Reviews_clean'], 
    tokenizer=LemmaTokenizer(), 
    vectorizer_type='tfidf', 
    ngram_range=(3, 3), 
    min_df=10, 
    max_features=500
)

**Logistic Regression with Trigrams and TfidfVectorizer**

In [36]:
accuracy_trigrams_tfidf, important_features_table_trigrams_tfidf = train_logistic_regression(
    x_train_trigrams_tfidf_vectorized, 
    y_train, 
    x_test_trigrams_tfidf_vectorized, 
    y_test, 
    trigram_tfidf_vectorizer, 
    top_n_features=50
)
important_features_table_trigrams_tfidf

Feature,Coefficient
can not save,-4.011161313152803
worst ever seen,-3.3818832386117954
one worst seen,-3.288704855309636
definitely worth watch,3.2552244024724666
highly recommend anyone,3.2368432158631433
not worth time,-3.236554961693414
really wanted like,-3.177912604526083
never get back,-3.157200250963703
not enough save,-3.144774493944503
waste time money,-3.035784370120819


### Testing Logistic Regression with TF-IDF Vectorizers: N-gram Range 1 to 3

**Vectorization with TfidfVectorizer (N-gram Range: 1 to 3)**

In [37]:
x_train_ngrams_tfidf_vectorized, x_test_ngrams_tfidf_vectorized, ngram_tfidf_vectorizer = vectorize_text(
    train_data['Reviews_clean'], 
    test_data['Reviews_clean'], 
    tokenizer=LemmaTokenizer(), 
    vectorizer_type='tfidf', 
    ngram_range=(1, 3), 
    min_df=10, 
    max_features=5000
)

### Feature Selection Using Chi-Squared Test for Unigrams, Bigrams, and Trigrams

In [42]:
from sklearn.feature_selection import chi2
import numpy as np
from prettytable import PrettyTable

N = 5000
featureselection = PrettyTable(["Unigram", "Bigram",  "Trigram"])

for category in y_train.unique():
    features_chi2 = chi2(x_train_ngrams_tfidf_vectorized, y_train == category)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(ngram_tfidf_vectorizer.get_feature_names_out())[indices]
    
    unigrams = [x for x in feature_names if len(x.split(' ')) == 1]
    bigrams = [x for x in feature_names if len(x.split(' ')) == 2]
    trigrams = [x for x in feature_names if len(x.split(' ')) == 3]
    
    for i in range(min(N, len(unigrams), len(bigrams), len(trigrams))):
        featureselection.add_row([
            unigrams[-(i+1)] if i < len(unigrams) else '',
            bigrams[-(i+1)] if i < len(bigrams) else '',
            trigrams[-(i+1)] if i < len(trigrams) else '',
        ])

featureselection

Unigram,Bigram,Trigram
worst,waste time,worst ever seen
great,not even,not waste time
bad,worst ever,one worst ever
waste,not waste,can not wait
awful,one worst,waste time money
terrible,one best,bad bad bad
boring,bad bad,worst ever made
loved,not worth,can not even
poor,really enjoyed,not wait see
worse,really bad,can not believe
